<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# xDeepFM : the eXtreme Deep Factorization Machine 
このノートブックでは、[xDeepFM モデル](https://arxiv.org/abs/1803.05170)をトレーニングする方法の簡単な例を示します。
xDeepFM [1] は、低次から高次のフィーチャ インタラクションの両方をキャプチャすることを目的とした、ディープラーニングベースのモデルを使用した精密なレコメンデーション システムです。したがって、フィーチャーの相互作用をより効果的に学習でき、手動でのフィーチャー エンジニアリングの労力を大幅に削減できます。要約すると、xDeepFM には次の主要な要素があります:
* CIN という名前のコンポーネントが含まれており、フィーチャの相互作用を明示的な方法でベクトル単位で学習します。
* 暗黙的な方法でビット単位でフィーチャーの相互作用を学習する、従来の DNN コンポーネントが含まれています。
* このモデルは非常に構成変更が可能な実装です。`use_Linear_part`、 `use_FM_part`、 `use_CIN_part` 及び `use_DNN_part` などのハイパーパラメーターを設定することで、コンポーネントの異なるサブセットを有効にできます。たとえば、`use_Linear_part` と `use_FM_part` のみを有効にするとで、古典的な FM モデルを取得できます。

このノートブックでは、1) 小さな合成データセットと 2) [Criteo データセット](http://labs.criteo.com/category/dataset)の 2 つのデータセットで xDeepFM をテストできます。

## 0. グローバル設定とインポート

In [1]:
import sys
sys.path.append("../../")
import os
import papermill as pm
from tempfile import TemporaryDirectory

import tensorflow as tf

from reco_utils.common.constants import SEED
from reco_utils.recommender.deeprec.deeprec_utils import (
    download_deeprec_resources, prepare_hparams
)
from reco_utils.recommender.deeprec.models.xDeepFM import XDeepFMModel
from reco_utils.recommender.deeprec.IO.iterator import FFMTextIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

tmpdir = TemporaryDirectory()

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Tensorflow version: 1.12.0


#### パラメータ

In [1]:
EPOCHS_FOR_SYNTHETIC_RUN = 15
EPOCHS_FOR_CRITEO_RUN = 30
BATCH_SIZE_SYNTHETIC = 128
BATCH_SIZE_CRITEO = 4096
RANDOM_SEED = SEED  # 非確定的な結果にする際には None を設定

xDeepFM はデータ入力として 次のようなFFM 形式を使用します: `<label> <field_id>:<feature_id>:<feature_value>`
各行はインスタンスを表し、`<label>` では 1 は正のインスタンスを意味し、0 は負のインスタンスを意味するバイナリ値です。
フィーチャはフィールドに分割されます。例えば、ユーザの性別はフィールドであり、男性、女性および不明の3つの選択可能な値を含みます。職業は、性別フィールドよりも多くの選択可能な値を含む別のフィールドにすることができます。フィールド インデックスとフィーチャ インデックスの両方が 1 から始まります。<br>

## 1.合成データ
それでは、小さな合成データセットから始めましょう。このデータセットには、10 のフィールド、1000 のフィーチャーがあり、一連の事前設定されたペアごとのフィーチャの相互作用の結果に従ってラベルが生成されます。

In [3]:
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'xDeepFM.yaml')
train_file = os.path.join(data_path, r'synthetic_part_0')
valid_file = os.path.join(data_path, r'synthetic_part_1')
test_file = os.path.join(data_path, r'synthetic_part_2')
output_file = os.path.join(data_path, r'output.txt')

if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/deeprec/', data_path, 'xdeepfmresources.zip')


10.6MB [00:00, 31.2MB/s]                            


#### 1.1 ハイパーパラメータの準備
prepare_hparams() は、学習率、フィーチャ数、ドロップアウト率など、モデルトレーニング用のハイパーパラメータの完全なセットを作成します。これらのパラメータを yaml ファイルに入れたり、関数のパラメーターとしてパラメータを渡したりできます (yaml 設定を上書きします)。

In [4]:
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=1000, 
                          FIELD_COUNT=10, 
                          cross_l2=0.0001, 
                          embed_l2=0.0001, 
                          learning_rate=0.001, 
                          epochs=EPOCHS_FOR_SYNTHETIC_RUN,
                          batch_size=BATCH_SIZE_SYNTHETIC)
print(hparams)

[('DNN_FIELD_NUM', None), ('FEATURE_COUNT', 1000), ('FIELD_COUNT', 10), ('MODEL_DIR', None), ('PAIR_NUM', None), ('SUMMARIES_DIR', None), ('activation', ['relu', 'relu']), ('attention_activation', None), ('attention_dropout', 0.0), ('attention_layer_sizes', None), ('batch_size', 128), ('cross_activation', 'identity'), ('cross_l1', 0.0), ('cross_l2', 0.0001), ('cross_layer_sizes', [1]), ('cross_layers', None), ('data_format', 'ffm'), ('dim', 10), ('doc_size', None), ('dropout', [0.0, 0.0]), ('dtype', 32), ('embed_l1', 0.0), ('embed_l2', 0.0001), ('enable_BN', False), ('entityEmb_file', None), ('entity_dim', None), ('entity_embedding_method', None), ('entity_size', None), ('epochs', 15), ('fast_CIN_d', 0), ('filter_sizes', None), ('init_method', 'tnormal'), ('init_value', 0.3), ('is_clip_norm', 0), ('iterator_type', None), ('kg_file', None), ('kg_training_interval', 5), ('layer_l1', 0.0), ('layer_l2', 0.0001), ('layer_sizes', [100, 100]), ('learning_rate', 0.001), ('load_model_name', 'yo

#### 1.2 データローダーの作成
モデルのデータ用イテレータを指定します。xDeepFM は FFMTextIterator を使用します。

In [5]:
input_creator = FFMTextIterator

#### 1.3 モデルの作成
ハイパーパラメータとデータ イテレータの両方の準備ができたら、モデルを作成できます。

In [6]:
model = XDeepFMModel(hparams, input_creator, seed=RANDOM_SEED)

## モデルを最初からトレーニングしたくない場合は、次のような
## 事前トレーニング済みのモデルを読み込むことができます:
#model.load_model(r'your_model_path')

Add CIN part.


/data/anaconda/envs/reco_gpu/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


次に、この時点でのモデルのパフォーマンスを見てみましょう (トレーニングを開始せずに):

In [7]:
print(model.run_eval(test_file))

{'auc': 0.5043, 'logloss': 0.7515}


AUC=0.5 はランダム推論の状態です。トレーニングの前に、モデルがランダムな推測のように振る舞うことが確認できます。

#### 1.4 モデルのトレーニング
次に、トレーニング セットでモデルをトレーニングし、検証データセットのパフォーマンスを確認します。モデルのトレーニングは、関数呼び出しと同じくらい簡単です:

In [8]:
model.fit(train_file, valid_file)

at epoch 1 train info: auc:0.5189, logloss:0.7006 eval info: auc:0.504, logloss:0.7042
at epoch 1 , train time: 3.0 eval time: 3.1
at epoch 2 train info: auc:0.5365, logloss:0.6919 eval info: auc:0.5066, logloss:0.6973
at epoch 2 , train time: 2.7 eval time: 3.1
at epoch 3 train info: auc:0.5552, logloss:0.6884 eval info: auc:0.5099, logloss:0.6953
at epoch 3 , train time: 2.8 eval time: 3.1
at epoch 4 train info: auc:0.5784, logloss:0.6848 eval info: auc:0.5147, logloss:0.6946
at epoch 4 , train time: 2.9 eval time: 3.1
at epoch 5 train info: auc:0.6104, logloss:0.6783 eval info: auc:0.523, logloss:0.6941
at epoch 5 , train time: 2.7 eval time: 3.2
at epoch 6 train info: auc:0.6582, logloss:0.6625 eval info: auc:0.5416, logloss:0.6929
at epoch 6 , train time: 2.7 eval time: 3.1
at epoch 7 train info: auc:0.7318, logloss:0.6204 eval info: auc:0.5916, logloss:0.6831
at epoch 7 , train time: 2.8 eval time: 3.1
at epoch 8 train info: auc:0.83, logloss:0.5231 eval info: auc:0.7024, logloss

#### 1.5 モデルの評価

ここでも、モデルのパフォーマンスを見てみましょう (トレーニング後):

In [9]:
res_syn = model.run_eval(test_file)
print(res_syn)
pm.record("res_syn", res_syn)

{'auc': 0.9716, 'logloss': 0.2278}


/data/anaconda/envs/reco_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.0). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  This is separate from the ipykernel package so we can avoid doing imports until


評価メトリックではなく完全な予測スコアを取得する場合は、次の操作で行うことができます:

In [10]:
model.predict(test_file, output_file)

## 2. Criteo データ 

これで、合成データセットの実験に成功しました。次に、実世界のデータセットとして [Criteo データセット](http://labs.criteo.com/category/dataset)の小さなサンプルで何かを試してみましょう。Criteo データセットは、CTR 予測モデルを開発するための業界のベンチマーク データセットとしてよく知られており、研究論文による評価データセットとして頻繁に採用されています。元のデータセットは軽量のデモでは大きすぎるため、デモ データセットとしてそのデータセットの一部をサンプリングします。

In [11]:
print('Criteo データセットによるデモ')
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=2300000, 
                          FIELD_COUNT=39, 
                          cross_l2=0.01, 
                          embed_l2=0.01, 
                          layer_l2=0.01,
                          learning_rate=0.002, 
                          batch_size=BATCH_SIZE_CRITEO, 
                          epochs=EPOCHS_FOR_CRITEO_RUN, 
                          cross_layer_sizes=[20, 10], 
                          init_value=0.1, 
                          layer_sizes=[20,20],
                          use_Linear_part=True, 
                          use_CIN_part=True, 
                          use_DNN_part=True)

train_file = os.path.join(data_path, r'cretio_tiny_train')
valid_file = os.path.join(data_path, r'cretio_tiny_valid')
test_file = os.path.join(data_path, r'cretio_tiny_test')

demo with Criteo dataset


In [12]:
model = XDeepFMModel(hparams, FFMTextIterator, seed=RANDOM_SEED)

# モデルをトレーニングする前に予測パフォーマンスを確認する
print(model.run_eval(test_file)) 
model.fit(train_file, valid_file)
# モデルのトレーニング後の予測パフォーマンスを確認する
res_real = model.run_eval(test_file)
print(res_real)
pm.record("res_real", res_real)

Add linear part.
Add CIN part.
Add DNN part.
{'auc': 0.4728, 'logloss': 0.7113}
at epoch 1 train info: auc:0.6648, logloss:0.5347 eval info: auc:0.6637, logloss:0.5342
at epoch 1 , train time: 27.9 eval time: 22.5
at epoch 2 train info: auc:0.7155, logloss:0.51 eval info: auc:0.7137, logloss:0.5109
at epoch 2 , train time: 24.3 eval time: 22.5
at epoch 3 train info: auc:0.7331, logloss:0.5012 eval info: auc:0.7283, logloss:0.5037
at epoch 3 , train time: 24.3 eval time: 22.2
at epoch 4 train info: auc:0.7412, logloss:0.4964 eval info: auc:0.7359, logloss:0.4991
at epoch 4 , train time: 24.1 eval time: 22.3
at epoch 5 train info: auc:0.7456, logloss:0.4937 eval info: auc:0.74, logloss:0.4963
at epoch 5 , train time: 24.1 eval time: 22.5
at epoch 6 train info: auc:0.7487, logloss:0.4919 eval info: auc:0.7426, logloss:0.4946
at epoch 6 , train time: 24.1 eval time: 22.3
at epoch 7 train info: auc:0.7505, logloss:0.4907 eval info: auc:0.7441, logloss:0.4934
at epoch 7 , train time: 24.2 ev

/data/anaconda/envs/reco_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.0). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  if __name__ == '__main__':


## 参考文献
\[1\] Lian, J., Zhou, X., Zhang, F., Chen, Z., Xie, X., & Sun, G. (2018). xDeepFM: Combining Explicit and Implicit Feature Interactions for Recommender Systems. Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery \& Data Mining, KDD 2018, London, UK, August 19-23, 2018.<br>

In [13]:
# クリーンアップ
tmpdir.cleanup()